# 작동 확인

In [1]:
import mysql.connector

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cur = remote.cursor()

remote.close()

# CSV 파일 열기

In [ ]:
import pandas as pd

df1801_1903 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_18.01~19.03.csv", encoding="cp949")
df1904_1905 = pd.read_excel("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_19.04~05.xlsx")
df1904_1905['대여일자'] = df1904_1905['대여일자'].astype(str)

df1906_1911 = pd.read_excel("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_19.06~19.11.xlsx")
df1906_1911.rename(columns={'대여일시' : '대여일자'}, inplace=True)
df1906_1911['대여일자'] = df1906_1911['대여일자'].astype(str)

#----------------- 아래는 고장난 파일 -----------------
# df1912_2005 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_19.12~20.05.csv", encoding="cp949")
# df2006_2006 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_20.06.01~06.30.csv", sep=',', encoding="EUC-KR")

df2007_2101 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_20.07~21.01.csv", encoding="cp949", usecols=[0,1])
df2007_2101.rename(columns={'대여일시' : '대여일자'}, inplace=True)

df2102_2106 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_21.02-06.csv", encoding="cp949", usecols=[0,1])
df2102_2106.rename(columns={'대여일시' : '대여일자'}, inplace=True)

df2107_2112 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_21.07-12.csv", encoding="cp949", usecols=[0,1])
df2107_2112.rename(columns={'대여일시' : '대여일자'}, inplace=True)

df2201_2206 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_22.01-06.csv", encoding="cp949", usecols=[0,1])
df2201_2206.rename(columns={'대여일시' : '대여일자'}, inplace=True)

df2207_2212= pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_22.07-12.csv", encoding="cp949", usecols=[0,1])
df2207_2212.rename(columns={'대여일시' : '대여일자'}, inplace=True)

df2301_2306 = pd.read_excel("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_23.01-06.xlsx")
df2301_2306.rename(columns={'대여일시' : '대여일자'}, inplace=True)
df2301_2306['대여일자'] = df2301_2306['대여일자'].astype(str)

df2307_2312 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_23.07-12.csv", encoding="cp949")
df2401_2406 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_24.01-06.csv", encoding="cp949")
df2407_2412 = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_24.07-12.csv", encoding="cp949")



df1904_1905.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   대여일자    61 non-null     object
 1   대여건수    61 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB


# concat결합 & 결측치 제거 & 전처리

In [101]:
df_daily_all = pd.concat([df1801_1903, df1904_1905, df1906_1911, df2007_2101, 
                          df2102_2106, df2107_2112, df2201_2206, df2207_2212, 
                          df2301_2306, df2307_2312, df2401_2406, df2407_2412])

df_daily_all = df_daily_all.dropna()

#---------------------대여건수 INT 만들기----------------------

rent_int = []

for idx, row in df_daily_all.iterrows():
    
    try:
        new = row['대여건수'].replace(',', '')
    except:
        new = row['대여건수']
    finally:    
        new = int(new)
        
    rent_int.append(new)


df_daily_all['대여건수'] = rent_int

df_daily_all

,대여일자,대여건수
0,2018-01-01,4950
1,2018-01-02,7136
2,2018-01-03,7156
3,2018-01-04,7102
4,2018-01-05,7705
...,...,...
179,2024-12-27,68333
180,2024-12-28,45732
181,2024-12-29,43614
182,2024-12-30,79482


# CSV 파일로 저장

In [102]:
df_daily_all.to_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_all.csv", 
                sep=",", index=False, encoding="utf-8")

# DB Table (rental_daily_summary_B) 만들기

In [3]:
remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)



cursor.execute("CREATE TABLE rental_daily_summary_B \
               (rental_date DATE, rental_count BIGINT NOT NULL)")

remote.commit()
remote.close()

# DB 에 집어넣기

In [104]:
import mysql.connector

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)

sql= """INSERT INto rental_daily_summary_B VALUES (%s, %s)"""

for i,row in df_daily_all.iterrows(): 

    date = row['대여일자']
    count = row['대여건수']

    cursor.execute(sql, (date, count))

    print(date, count)

    
remote.commit()
remote.close()

2018-01-01 4950
2018-01-02 7136
2018-01-03 7156
2018-01-04 7102
2018-01-05 7705
2018-01-06 5681
2018-01-07 5220
2018-01-08 6309
2018-01-09 5988
2018-01-10 4476
2018-01-11 4337
2018-01-12 4401
2018-01-13 3756
2018-01-14 4675
2018-01-15 6993
2018-01-16 7421
2018-01-17 6990
2018-01-18 7054
2018-01-19 8329
2018-01-20 6148
2018-01-21 5574
2018-01-22 4929
2018-01-23 4263
2018-01-24 3370
2018-01-25 3307
2018-01-26 3116
2018-01-27 2833
2018-01-28 3028
2018-01-29 4425
2018-01-30 3591
2018-01-31 4104
2018-02-01 5821
2018-02-02 6557
2018-02-03 3499
2018-02-04 2642
2018-02-05 4213
2018-02-06 4257
2018-02-07 5028
2018-02-08 6114
2018-02-09 6905
2018-02-10 4829
2018-02-11 3047
2018-02-12 5231
2018-02-13 6863
2018-02-14 7477
2018-02-15 3935
2018-02-16 3047
2018-02-17 3494
2018-02-18 5191
2018-02-19 8563
2018-02-20 8899
2018-02-21 8342
2018-02-22 8580
2018-02-23 6408
2018-02-24 7198
2018-02-25 7595
2018-02-26 10712
2018-02-27 10064
2018-02-28 4230
2018-03-01 5761
2018-03-02 9388
2018-03-03 16029
2018-

---

# CSV (df_rental) 불러오기 

In [6]:
import pandas as pd

df_rental = pd.read_csv("/home/dan/dev_ws/EDA/project/data/Rental/rent_daily_station.csv", sep=",", encoding="utf-8")

df_rental

,대여소ID,날짜,대여량
0,ST-10,2024-01-01,35
1,ST-10,2024-01-02,38
2,ST-10,2024-01-03,20
3,ST-10,2024-01-04,31
4,ST-10,2024-01-05,27
...,...,...,...
984374,ST-997,2024-12-27,105
984375,ST-997,2024-12-28,62
984376,ST-997,2024-12-29,53
984377,ST-997,2024-12-30,101


# DB Table (rental_daily_station) 만들기

In [15]:
import mysql.connector

remote = mysql.connector.connect(
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)

cursor.execute("CREATE TABLE rental_daily_station \
               (rental_station_id VARCHAR(10) NOT NULL, rental_date DATE, rental_count BIGINT NOT NULL)")

remote.commit()
remote.close()

# DB (rental_daily_station)에 집어넣기

In [16]:
import mysql.connector
from tqdm import tqdm

remote = mysql.connector.connect (
    host = "jinhyuk2db.c94oygiiukcg.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = "root",
    password = "jinhyuk2dacibul",
    database = "project1"
)

cursor = remote.cursor(buffered=True)

sql= """INSERT INto rental_daily_station VALUES (%s, %s, %s)"""

for i,row in tqdm(df_rental.iterrows(), total=len(df_rental)): 

    id = row['대여소ID']
    date = row['날짜']
    count = row['대여량']

    cursor.execute(sql, (id, date, count))

    # print(id, date, count)

    
remote.commit()
remote.close()

100%|██████████| 984379/984379 [3:11:42<00:00, 85.58it/s]   
